In [288]:
#import data
import pandas as pd
import numpy as np

import glob


#transfer_plate=pd.read_csv("test/transfer_plate.csv",header=0)
arr = pd.read_csv("test/transfer_plate.csv").iloc[:, 1:-4].to_numpy()
transfer_plate_labels=pd.read_csv("test/transfer_plate.csv").iloc[:, -4:].to_numpy()
test_data = pd.read_csv("test/96_samples.csv",header=None).iloc[:, 1:].to_numpy()

csv_files = glob.glob("train/*.csv")
dataframes = {}
for file in csv_files:
    
    dataframes[file] = pd.read_csv(file)
    


transfer_plate_labels#here i will assume that the these samples are well placed
#i.e. the sample number corresponds to the actual sample and the order isn't mantained

array([['sample1', 4.61928175, 1.937171963, 1.052928098],
       ['sample2', 5.782717628, 1.175901957, 1.214737524],
       ['sample3', 3.953447633, 1.350473236, 2.132459339],
       ['sample4', 2.038083652, 0.948045427, 1.380239621],
       ['sample5', 4.978294553, 0.459765296, 2.539621518],
       ['sample6', 4.262552834, 0.336063472, 1.55097026],
       ['sample7', 7.819111584, 0.846057989, 0.785458041],
       ['sample8', 9.390954184, 1.36481141, 1.676221842],
       ['sample9', 6.713935235, 1.853940478, 2.054206282],
       ['sample10', 8.779337413, 1.361967507, 1.81236406],
       ['sample11', 8.605301328, 2.056711184, 0.823049343],
       ['sample12', 11.47050932, 1.445822976, 2.123401956],
       ['sample13', 10.62416263, 0.980751729, 1.429506685],
       ['sample14', 8.608632725, 0.456044771, 1.905153587],
       ['sample15', 4.775166356, 1.56874649, 1.777716209],
       ['sample16', 5.416822914, 1.384659314, 1.306419633],
       ['sample17', 5.98264914, 2.058122288, 2.4440160

In [289]:
transfer_plate_labels_1=[]
for i in range(len(transfer_plate_labels)-1):
    
    transfer_plate_labels_1.append(transfer_plate_labels[i])
    transfer_plate_labels_1.append(transfer_plate_labels[i])
  
transfer_plate_labels_1=np.array(transfer_plate_labels_1)[:192,-3:]
transfer_plate_labels_1=np.array(transfer_plate_labels_1,dtype=np.float32)

#oogga boogga data handling

In [290]:
from scipy.interpolate import interp1d

target_wavenumbers = np.linspace(300, 1942, 1340)
interpolated = []
conc=[]

for file in csv_files:
    # Convert column names to float
    print(file)
    wavelengths = dataframes[file].columns[:-5].astype(float)
    # Get only the spectral data
    intensities_y = (dataframes[file].values)

    conc.append(intensities_y[:,-5:-2])
    #normalization steps --1-- scaling
    
    intensities_y=intensities_y/(np.max(intensities_y))

    print(intensities_y)

    # Interpolate each row (spectrum)
    
    for row in intensities_y:
        
        f = interp1d(wavelengths, row[:-5], kind='quadratic', fill_value='extrapolate')

        interpolated.append(f(target_wavenumbers))
        
    

interpolated=np.array(interpolated)

#normalization step --2--

interpolated=(interpolated-np.mean(interpolated))/np.std(interpolated)

conc=np.vstack(conc)



train\anton_532.csv
[[6.12906223e-01 6.18048921e-01 6.01548923e-01 ... 1.17462023e-04
  0.00000000e+00 0.00000000e+00]
 [6.04361960e-01 6.11595483e-01 5.97326514e-01 ... 1.17462023e-04
  0.00000000e+00 0.00000000e+00]
 [6.01281558e-01 6.09306579e-01 5.95375892e-01 ... 1.17462023e-04
  0.00000000e+00 0.00000000e+00]
 ...
 [6.31933961e-01 6.37724692e-01 6.22076512e-01 ... 2.83731990e-04
  0.00000000e+00 3.26054705e-04]
 [6.32346420e-01 6.36125394e-01 6.20006880e-01 ... 2.83731990e-04
  0.00000000e+00 3.26054705e-04]
 [6.29817051e-01 6.35774885e-01 6.20083503e-01 ... 2.83731990e-04
  0.00000000e+00 3.26054705e-04]]
train\anton_785.csv
[[3.30425459e-01 3.28524414e-01 3.25652804e-01 ... 5.27940930e-07
  0.00000000e+00 0.00000000e+00]
 [3.30246520e-01 3.26422957e-01 3.23595724e-01 ... 5.27940930e-07
  0.00000000e+00 0.00000000e+00]
 [3.27078588e-01 3.25612722e-01 3.24311479e-01 ... 5.27940930e-07
  0.00000000e+00 0.00000000e+00]
 ...
 [5.16342835e-01 4.81799065e-01 4.48261646e-01 ... 4.98278

Now we standardize the weird ones


In [295]:
arr = np.char.strip(arr.astype(str), "[]").astype(float)
test_data = np.char.strip(test_data.astype(str),"[]").astype(float)

# normalization step --- 1 --- scaling
arr=arr/np.max(arr)

test_data=test_data/np.max(test_data)



In [296]:
wavelengths=np.linspace(65,3350,2048)#make sure this is the actual interval.

#interpolate test_data and the transfer_plate datasets(same format)
transfer_plate=[]
clean_test=[]
for row in arr:

    f = interp1d(wavelengths, row, kind='quadratic', fill_value='extrapolate')
    transfer_plate.append(f(target_wavenumbers))

for row in test_data:

    g = interp1d(wavelengths, row, kind='quadratic', fill_value='extrapolate')
    
    clean_test.append(g(target_wavenumbers))


transfer_plate=(transfer_plate-np.mean(transfer_plate))/np.std(transfer_plate)
clean_test=(clean_test-np.mean(clean_test))/np.std(clean_test)


transfer_plate=np.array(transfer_plate,dtype=np.float32)

In [297]:
from Raman_Dataset import RamanDataset
from torch.utils.data import DataLoader , ConcatDataset





dataset1 = RamanDataset(transfer_plate, transfer_plate_labels_1)
dataset2= RamanDataset(interpolated,conc)

#concatenate datasets
dataset= ConcatDataset([dataset1,dataset2])

loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Example: iterate
for spectra_batch, targets_batch in loader:
    print(spectra_batch.shape)  # (batch_size, 1, num_points)
    
    print(targets_batch.shape)  # (batch_size, 3)
    break

torch.Size([32, 1, 1340])
torch.Size([32, 3])


In [ ]:
from SimpleRamanCNN import SimpleRamanCNN
import torch
import torch.nn as nn

model = SimpleRamanCNN()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.MSELoss()

epochs=100

for epoch in range(epochs):
    running_loss=0
    print(epoch)
    for X_batch, y_batch in loader:
        optimizer.zero_grad()
        preds = model(X_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch}: Loss {running_loss/len(loader)}")

0
Epoch 0: Loss 6.226226676594127
1
Epoch 1: Loss 5.942112919572112
2
Epoch 2: Loss 6.15941706570712
3
Epoch 3: Loss 5.882156396841074
4
Epoch 4: Loss 5.8679123698890985
5
Epoch 5: Loss 5.875291762413917
6
Epoch 6: Loss 5.8178322036545
7
Epoch 7: Loss 5.831007229817378
8
Epoch 8: Loss 5.841203847488799
9
Epoch 9: Loss 5.831352633315247
10
Epoch 10: Loss 5.8561629722644755
11
Epoch 11: Loss 5.851954261977951
12
Epoch 12: Loss 5.823033023190189
13
Epoch 13: Loss 5.83346853627787
14
Epoch 14: Loss 5.836355357975155
15
Epoch 15: Loss 5.786482024502445
16
Epoch 16: Loss 5.790514475339419
17
Epoch 17: Loss 5.8361517392195665
18
Epoch 18: Loss 5.790703039664726
19
Epoch 19: Loss 5.784494926403095
20
Epoch 20: Loss 5.7754391571144
21
Epoch 21: Loss 5.741370749163937
22
Epoch 22: Loss 5.7112406910239875
23
Epoch 23: Loss 5.727572537087775
24
Epoch 24: Loss 5.67599716434231
25
Epoch 25: Loss 5.660339058219613
26
Epoch 26: Loss 5.68980022529503
27
Epoch 27: Loss 5.633326694562838
28
Epoch 28: Los

In [ ]:
clean_test=np.array(clean_test,dtype=np.float32)



In [ ]:
test_loader = DataLoader(RamanDataset(clean_test,None, train=False), batch_size=32, shuffle=False)
sanity_check= DataLoader(RamanDataset(interpolated,None, train=False), batch_size=32, shuffle=False)

model.eval()  # switch to evaluation mode
predictions = []

with torch.no_grad():
    for spectra in test_loader:
        outputs = model(spectra)  # shape (batch, 3)
        predictions.append(outputs)

predictions = torch.cat(predictions).numpy()
predictions

array([[8.034806 , 1.3838711, 1.8652562],
       [8.056719 , 1.3867826, 1.869138 ],
       [7.4951944, 1.3138247, 1.7686137],
       [7.484438 , 1.3123448, 1.7668195],
       [8.073463 , 1.3889196, 1.8722447],
       [8.087784 , 1.3907633, 1.8748627],
       [8.025521 , 1.3827238, 1.863549 ],
       [8.028694 , 1.3830872, 1.8642291],
       [7.883113 , 1.3643456, 1.838196 ],
       [7.88231  , 1.3642464, 1.8380041],
       [8.146275 , 1.3984494, 1.8855335],
       [8.168219 , 1.4012535, 1.8895187],
       [7.891885 , 1.3654449, 1.8395822],
       [7.783639 , 1.3513566, 1.8202593],
       [8.132521 , 1.3965974, 1.8831689],
       [8.169794 , 1.4013925, 1.8899357],
       [6.276248 , 1.1561089, 1.5500989],
       [6.275688 , 1.156003 , 1.5501238],
       [8.062359 , 1.3877294, 1.8702607],
       [8.259164 , 1.4133382, 1.9055871],
       [6.070451 , 1.1298339, 1.5132535],
       [6.107667 , 1.1346686, 1.5198712],
       [5.9858484, 1.1188934, 1.498081 ],
       [6.0004387, 1.1207752, 1.50

In [ ]:
#load predictions to sample_submission.csv

template= pd.read_csv("test/sample_submission.csv")


averaged = predictions.reshape(-1, 2, 3).mean(axis=1)

template[['Glucose', 'Sodium Acetate', 'Magnesium Sulfate']] = averaged

# Save to new CSV
template.to_csv("test/sample_submission.csv", index=False)

Now we apply CV to fine tune hiperparameters

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import KFold

# === Your Dataset Class ===
class RamanDataset(Dataset):
    def __init__(self, spectra, targets, augment=False):
        self.spectra = spectra  # Tensor: shape [N, length]
        self.targets = targets  # Tensor: shape [N]
        self.augment = augment

    def __len__(self):
        return len(self.spectra)

    def __getitem__(self, idx):
        spectrum = self.spectra[idx]
        label = self.targets[idx]

        if self.augment:
            spectrum = spectrum + sinusoidal_noise(len(spectrum))  # your noise fn
            spectrum = left_right_shift(torch.linspace(300,1940,1340), spectrum, maxshift=5)  # example

        spectrum = spectrum.unsqueeze(0)

        return spectrum, label


# === K-fold cross validation setup ===
def run_kfold_cv(spectra, targets, k=5, batch_size=32, epochs=10):
    kfold = KFold(n_splits=k, shuffle=True, random_state=42)
    all_val_losses = []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(spectra)):
        print(f"\n---- Fold {fold+1}/{k} ----")


        train_dataset = RamanDataset(spectra[train_idx], targets[train_idx], augment=True)
        val_dataset   = RamanDataset(spectra[val_idx], targets[val_idx], augment=False)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        # Create model
        model = SimpleRamanCNN()  # replace with your model class
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        criterion = torch.nn.CrossEntropyLoss()

        # Training loop
        for epoch in range(epochs):
            model.train()
            for X, y in train_loader:
                optimizer.zero_grad()
                out = model(X)
                loss = criterion(out, y)
                loss.backward()
                optimizer.step()

            # Validation loop
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for X, y in val_loader: 
                    out = model(X)
                    val_loss += criterion(out, y).item()

            avg_val_loss = val_loss / len(val_loader)
            print(f"Epoch {epoch+1}/{epochs} - Val Loss: {avg_val_loss:.4f}")

        all_val_losses.append(avg_val_loss)

    print(f"\nAverage Val Loss across folds: {sum(all_val_losses)/len(all_val_losses):.4f}")


# === Example call ===
# spectra, targets = your Raman data as tensors
# wavelengths = your x-axis tensor for augmentation

spectra = torch.tensor(interpolated, dtype=torch.float32)
targets = torch.tensor(conc, dtype=torch.float32)
run_kfold_cv(spectra, targets, k=5, batch_size=32, epochs=10)


---- Fold 1/5 ----
Epoch 1/10 - Val Loss: 4.2164
Epoch 2/10 - Val Loss: 4.2069
Epoch 3/10 - Val Loss: 4.2054
Epoch 4/10 - Val Loss: 4.1948
Epoch 5/10 - Val Loss: 4.2072
Epoch 6/10 - Val Loss: 4.1961
Epoch 7/10 - Val Loss: 4.2173
Epoch 8/10 - Val Loss: 4.2038


KeyboardInterrupt: 